Analyzing Mind Brain Body Wave 1 sequences. Amplicon sequencing of the V4 region of the 16S gene was performed with the 515f/806r primer set (Caporaso et al., 2011) following Earth Microbiome Project (EMP protocol).

We are using an open source software called Qiime2. Qiime2 is useful for beginners to 16S research because it is well-documented, with extensive online tutorials, and user-friendly. See docs.qiime2.org for more information.

# Import

We are using the Casava 1.8 paired-end FastQ protocol from Qiime2 documentation: https://docs.qiime2.org/2022.2/tutorials/importing/#importing-seqs.

Importing lets us create, from the sequences, a Qiime2 "artifact", ie, an object that is manipulable in Qiime2.

In [2]:
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path 5064046_all_16S/fastq \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux-paired-end.qza

Imported 5064046_all_16S/fastq as CasavaOneEightSingleLanePerSampleDirFmt to demux-paired-end.qza


Now, we can visualize the data to make sure it imported correctly.

In [4]:
!qiime demux summarize\
    --i-data demux-paired-end.qza\
    --o-visualization demux-paired-end.qzv

Saved Visualization to: demux-paired-end.qzv


We can view the imported sequences by uploading the file to view.qiime2.org. We have 7696724 each of forward and reverse reads.

# Denoising

Note: these sequences do *not* contain primers that would need to be trimmed, so we can proceed to quality control.

Dada2 is a pipeline that includes various quality control steps, including merging the paired ends, removing chimeric sequences, etc. 

In [1]:
! qiime dada2 denoise-paired \
  --i-demultiplexed-seqs demux-paired-end.qza \
  --p-trunc-len-f 230 \
  --p-trunc-len-r 230 \
  --p-trim-left-r 15 \
  --p-n-threads 0 \
  --o-table feature-table.qza \
  --o-representative-sequences rep-seqs.qza \
  --o-denoising-stats denoising-stats.qza

Saved FeatureTable[Frequency] to: feature-table.qza
Saved FeatureData[Sequence] to: rep-seqs.qza
Saved SampleData[DADA2Stats] to: denoising-stats.qza


In [2]:
# Create visualizations for de-noising artifacts outputted above
! qiime feature-table summarize \
  --i-table feature-table.qza \
  --o-visualization feature-table.qzv \
  --m-sample-metadata-file mbb_saliva_metadata_w1.tsv

! qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv

! qiime metadata tabulate \
  --m-input-file denoising-stats.qza \
  --o-visualization denoising-stats.qzv

Saved Visualization to: feature-table.qzv
Saved Visualization to: rep-seqs.qzv
Saved Visualization to: denoising-stats.qzv


# Phylogeny

In [3]:
# Alignment of representative sequences using MAFFT (ie, infer homology)
! qiime alignment mafft \
  --i-sequences rep-seqs.qza \
  --o-alignment aligned-rep-seqs.qza

# Mask alignments (eliminate ambiguous alignments)
! qiime alignment mask \
  --i-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza

# Construct phylogeny using fasttree
! qiime phylogeny fasttree \
  --i-alignment masked-aligned-rep-seqs.qza \
  --o-tree fasttree-tree.qza

# Root the tree
! qiime phylogeny midpoint-root \
  --i-tree fasttree-tree.qza \
  --o-rooted-tree fasttree-tree-rooted.qza

Saved FeatureData[AlignedSequence] to: aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: masked-aligned-rep-seqs.qza
Saved Phylogeny[Unrooted] to: fasttree-tree.qza
Saved Phylogeny[Rooted] to: fasttree-tree-rooted.qza


# Taxonomy

Refer to this tutorial https://docs.qiime2.org/2022.2/tutorials/feature-classifier/. For taxonomic assignment, we train a naive bayes classifier on reference sequences/taxonomies. Once the classifier is trained, we can use it to classify our sequences.

Our reference database is Silva, as this is very widely used. The version is 138.1, downloaded from: https://docs.qiime2.org/2022.2/data-resources/. Another database we could use is Human Oral Microbiome Database (HOMD). HOMD will have the benefit of including only sequences we'd expect to find in the human oral microbiome, and training the classifier takes less time than Silva. However, Silva is more widely used and has more sequences overall. In a previous project, I found the taxonomic assignment from HOMD- and Silva-trained classifiers to be comparable.

Another option is to use a "bespoke" bayes classifier, which is weighted to favor sequences we expect to see in this habitat. Many aspects of this are still in development. However, weights are available here: https://github.com/BenKaehler/readytowear; weight used here is this one: https://github.com/BenKaehler/readytowear/blob/master/data/silva_138_1/515f-806r/human-oral.qza and we use reference sequence/taxonomy from the same folder. These are created using deblur outputs of length 150bp, so slightly different from our own samples. We ran both and compared the resulting resolution before choosing. For the benefit of using a bespoke classifier, see: https://www.nature.com/articles/s41467-019-12669-6.

The default k-mer length for the classifier is 7. Per Bokulich et al. 2018 (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5956843/), I'm using a k-mer length of 12.

In [5]:
#Extract V4 reads from database using the primers used for our data
#The min/max length restrictions are to reduce sequences of very unusual length
!qiime feature-classifier extract-reads \
  --i-sequences silva-138-99-seqs.qza \ #from qiime2
  --p-f-primer GTGCCAGCMGCCGCGGTAA \
  --p-r-primer GGACTACHVGGGTWTCTAAT \
  --p-min-length 150 \
  --p-max-length 350 \
  --o-reads ref-seqs.qza

Saved FeatureData[Sequence] to: ref-seqs.qza


In [11]:
#Train bespoke classifier with k=mer length 12 and human oral class weights
! qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads ref-seqs_readytowear.qza \
  --i-reference-taxonomy ref-tax_readytowear.qza \
  --i-class-weight human-oral.qza \
  --p-feat-ext--ngram-range '[12,12]' \
  --o-classifier nb-classifier-bespoke.qza

Saved TaxonomicClassifier to: nb-classifier-bespoke.qza


In [4]:
#Assign taxonomy using bespoke classifier
! qiime feature-classifier classify-sklearn \
  --i-classifier nb-classifier-bespoke.qza \
  --i-reads rep-seqs.qza \
  --o-classification bespoke-taxonomy.qza

! qiime metadata tabulate \
  --m-input-file bespoke-taxonomy.qza \
  --o-visualization bespoke-taxonomy.qzv

Saved FeatureData[Taxonomy] to: bespoke-taxonomy.qza
Saved Visualization to: bespoke-taxonomy.qzv




Using the visualization below, we'll determine an appropriate sampling depth for rarefaction.

In [5]:
!qiime diversity alpha-rarefaction \
  --i-table feature-table.qza \
  --o-visualization alpha_rarefaction_curves_.qzv \
  --p-min-depth 10 \
  --p-max-depth 20000

Saved Visualization to: alpha_rarefaction_curves_.qzv


We can rarefy to 14,697 which will include all samples.

In [6]:
# Create rarefied feature table
! qiime feature-table rarefy \
  --i-table feature-table.qza \
  --p-sampling-depth 14697 \
  --o-rarefied-table 14697-feat-table.qza

#Taxa barplot (non-rarefied) for R analyses
##No metadata because it becomes columns in the downloaded CSV
##Note: no longer using this; using Biom export for diff abundance instead
#! qiime taxa barplot \
#  --i-table feature-table.qza \
#  --i-taxonomy bespoke-taxonomy.qza \
#  --o-visualization taxa-barplot.qzv

#Taxa barplot (rarefied) for visuals
! qiime taxa barplot \
  --i-table 14697-feat-table.qza \
  --i-taxonomy bespoke-taxonomy.qza \
  --m-metadata-file mbb_saliva_metadata_w1.tsv \
  --o-visualization 14697-taxa-barplot.qzv

Saved FeatureTable[Frequency] to: 14697-feat-table.qza
Saved Visualization to: 14697-taxa-barplot.qzv


# Analyses

In [2]:
#this is rarefied to the highest depth where we can still keep all samples
!qiime diversity core-metrics-phylogenetic \
  --i-table feature-table.qza \
  --i-phylogeny fasttree-tree-rooted.qza \
  --m-metadata-file mbb_saliva_metadata_w1.tsv \
  --p-sampling-depth 14697 \
  --output-dir qiime-diversity-results

Saved FeatureTable[Frequency] to: qiime-diversity-results/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: qiime-diversity-results/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: qiime-diversity-results/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: qiime-diversity-results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: qiime-diversity-results/evenness_vector.qza
Saved DistanceMatrix to: qiime-diversity-results/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: qiime-diversity-results/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: qiime-diversity-results/jaccard_distance_matrix.qza
Saved DistanceMatrix to: qiime-diversity-results/bray_curtis_distance_matrix.qza
Saved PCoAResults to: qiime-diversity-results/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: qiime-diversity-results/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: qiime-diversity-results/jaccard_pcoa_results.qza
Saved PCoAResults to: qiim

In [3]:
#Export Level 6 table
##Collapse to genus level
!qiime taxa collapse \
  --i-table feature-table.qza \
  --i-taxonomy bespoke-taxonomy.qza \
  --p-level 6 \
  --o-collapsed-table genus_table.qza

##Export to biom directory
!qiime tools export \
  --input-path genus_table.qza \
  --output-path exported-genus-table

##Convert biom to tsv
!biom convert -i exported-genus-table/feature-table.biom -o genus-table.tsv --to-tsv


#Export Level 7 table
##Collapse to species level
!qiime taxa collapse \
  --i-table feature-table.qza \
  --i-taxonomy bespoke-taxonomy.qza \
  --p-level 7 \
  --o-collapsed-table species_table.qza

##Export to biom directory
!qiime tools export \
  --input-path species_table.qza \
  --output-path exported-species-table

##Convert biom to tsv
!biom convert -i exported-species-table/feature-table.biom -o species-table.tsv --to-tsv


#Export ASV Table
!qiime tools export \
  --input-path feature-table.qza \
  --output-path exported-asv-table

##Convert biom to tsv
!biom convert -i exported-asv-table/feature-table.biom -o asv-table.tsv --to-tsv

Saved FeatureTable[Frequency] to: genus_table.qza
Exported genus_table.qza as BIOMV210DirFmt to directory exported-genus-table
Saved FeatureTable[Frequency] to: species_table.qza
Exported species_table.qza as BIOMV210DirFmt to directory exported-species-table
Exported feature-table.qza as BIOMV210DirFmt to directory exported-asv-table


In [1]:
#Export Level 2 table
##Collapse to genus level
!qiime taxa collapse \
  --i-table feature-table.qza \
  --i-taxonomy bespoke-taxonomy.qza \
  --p-level 2 \
  --o-collapsed-table phylum_table.qza

##Export to biom directory
!qiime tools export \
  --input-path phylum_table.qza \
  --output-path exported-phylum-table

##Convert biom to tsv
!biom convert -i exported-phylum-table/feature-table.biom -o phylum-table.tsv --to-tsv


#Export Level 3 table
##Collapse to genus level
!qiime taxa collapse \
  --i-table feature-table.qza \
  --i-taxonomy bespoke-taxonomy.qza \
  --p-level 3 \
  --o-collapsed-table class_table.qza

##Export to biom directory
!qiime tools export \
  --input-path class_table.qza \
  --output-path 'exported-class-table'

##Convert biom to tsv
!biom convert -i 'exported-class-table/feature-table.biom' -o 'class-table.tsv' --to-tsv


#Export Level 4 table
##Collapse to genus level
!qiime taxa collapse \
  --i-table feature-table.qza \
  --i-taxonomy bespoke-taxonomy.qza \
  --p-level 4 \
  --o-collapsed-table order_table.qza

##Export to biom directory
!qiime tools export \
  --input-path order_table.qza \
  --output-path exported-order-table

##Convert biom to tsv
!biom convert -i exported-order-table/feature-table.biom -o order-table.tsv --to-tsv


#Export Level 5 table
##Collapse to genus level
!qiime taxa collapse \
  --i-table feature-table.qza \
  --i-taxonomy bespoke-taxonomy.qza \
  --p-level 5 \
  --o-collapsed-table family_table.qza

##Export to biom directory
!qiime tools export \
  --input-path family_table.qza \
  --output-path exported-family-table

##Convert biom to tsv
!biom convert -i exported-family-table/feature-table.biom -o family-table.tsv --to-tsv

Saved FeatureTable[Frequency] to: phylum_table.qza
Exported phylum_table.qza as BIOMV210DirFmt to directory exported-phylum-table
Saved FeatureTable[Frequency] to: class_table.qza
Exported class_table.qza as BIOMV210DirFmt to directory exported-class-table
Saved FeatureTable[Frequency] to: order_table.qza
Exported order_table.qza as BIOMV210DirFmt to directory exported-order-table
Saved FeatureTable[Frequency] to: family_table.qza
Exported family_table.qza as BIOMV210DirFmt to directory exported-family-table
